<a href="https://colab.research.google.com/github/bhavanapagadala/hackton/blob/main/disaster_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install folium geopy ipywidgets

import folium
from geopy.geocoders import Nominatim
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# 📘 Display Feature Info Above Map
info_html = HTML("""
<h3>🌐 Disaster Response Features</h3>
<ul>
  <li>🚨 <b>Disaster Location</b> — marked in red</li>
  <li>✅ <b>Safe Zone</b> — nearby escape area</li>
  <li>🔵 <b>Evacuation Route</b> — blue line to safety</li>
  <li>🏥 <b>Hospitals</b> — for medical support</li>
  <li>🛟 <b>Rescue Centers</b> — emergency responders</li>
</ul>
""")

# Widgets for input
search_box = widgets.Text(
    value='',
    placeholder='Enter any Indian city/location',
    description='🔍 Search:',
    layout=widgets.Layout(width='75%')
)
search_button = widgets.Button(description='Search & Plot', button_style='success')
output = widgets.Output()

geolocator = Nominatim(user_agent="india_disaster_map")

# Action when button is clicked
def on_search_clicked(b):
    with output:
        clear_output()
        location_query = search_box.value.strip()
        if not location_query:
            print("❌ Please enter a location.")
            return

        location = geolocator.geocode(f"{location_query}, India")
        if not location:
            print("❌ Location not found.")
            return

        lat, lon = location.latitude, location.longitude
        print(f"✅ Found: {location.address}")

        # Create map
        m = folium.Map(location=[lat, lon], zoom_start=13)

        # 🚨 Disaster Location
        folium.Marker(
            [lat, lon],
            tooltip=f"🚨 Disaster at {location_query}",
            popup=folium.Popup(f"<b>Severity:</b> Critical<br><b>Rescue Contact:</b> 100", max_width=250),
            icon=folium.Icon(color="darkred", icon='exclamation-sign')
        ).add_to(m)

        # ✅ Safe Zone (relative offset)
        safe_lat, safe_lon = lat - 0.01, lon - 0.01
        folium.Marker(
            [safe_lat, safe_lon],
            tooltip="✅ Safe Zone",
            popup="Community Ground (Safe Zone)",
            icon=folium.Icon(color="green", icon='ok-sign')
        ).add_to(m)

        # 🔵 Evacuation Route
        folium.PolyLine(
            locations=[[lat, lon], [safe_lat, safe_lon]],
            color="blue", weight=3, tooltip="Evacuation Route"
        ).add_to(m)

        # 🏥 Hospitals (offsets)
        hospitals = [
            {"name": "City Hospital", "lat": lat + 0.015, "lon": lon + 0.005},
            {"name": "General Hospital", "lat": lat - 0.015, "lon": lon - 0.008}
        ]
        for h in hospitals:
            folium.Marker(
                [h["lat"], h["lon"]],
                tooltip=f"🏥 {h['name']}",
                popup=h["name"],
                icon=folium.Icon(color="lightblue", icon="plus-sign")
            ).add_to(m)

        # 🛟 Rescue Centers (offsets)
        rescue_centers = [
            {"name": "Fire & Rescue HQ", "lat": lat + 0.01, "lon": lon - 0.01},
            {"name": "Coast Guard Base", "lat": lat - 0.012, "lon": lon + 0.01}
        ]
        for r in rescue_centers:
            folium.Marker(
                [r["lat"], r["lon"]],
                tooltip=f"🛟 {r['name']}",
                popup=r["name"],
                icon=folium.Icon(color="purple", icon="flag")
            ).add_to(m)

        # Show info + map
        display(info_html)
        display(m)

# Connect button click
search_button.on_click(on_search_clicked)

# Show search UI
display(widgets.HBox([search_box, search_button]))
display(output)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.9 MB/s eta 0:00:00


Output()